# Python Example for FPGA Lesson 4
This notebook shows how to interface the AXI memory mapped FPGA lesson 4 bitstream using Python.

https://redpitaya-knowledge-base.readthedocs.io/en/latest/learn_fpga/4_lessons/StopWatch.html  
http://antonpotocnik.com/?p=489265

### Load the Stopwatch bitstream
The two cells below load the bitstream for the stopwatch example and demonstrate it.

The memory map will close as part of garbage collection

In [1]:
import os
os.system('cat /root/stopwatch.bit > /dev/xdevcfg')

0

In [2]:
import mmap
import os
import time
import numpy as np

axi_gpio_regset = np.dtype([
    ('gpio1_data'   , 'uint32'),
    ('gpio1_control', 'uint32'),
    ('gpio2_data'   , 'uint32'),
    ('gpio2_control', 'uint32')
])

memory_file_handle = os.open('/dev/mem', os.O_RDWR)
axi_mmap = mmap.mmap(fileno=memory_file_handle, length=mmap.PAGESIZE, offset=0x40000000)
axi_numpy_array = np.recarray(1, axi_gpio_regset, buf=axi_mmap)
axi_array_contents = axi_numpy_array[0]

freq = 124998750 #FPGA Clock Frequency Hz

In [9]:
axi_array_contents.gpio1_data = 0x02 #clear timer
axi_array_contents.gpio1_data = 0x01 #start timer
time.sleep(34.2) # Count to the maximim LED (8 MSB value)
axi_array_contents.gpio1_data = 0x00 #stop timer
print("Clock count: ", axi_array_contents.gpio2_data, " calculated time: ", axi_array_contents.gpio2_data/freq, " Seconds")

Clock count:  4278907400  calculated time:  1711.56296  Seconds


In [4]:
os.system('cat /root/stopwatch.bit > /dev/xdevcfg')

0

### Change Clock Speed
The following terminal commands can be used to change the PL fabric clock speed. The script needs root access.
The clock frequancy can be set from 100000 to 2500000000. Above 300000 gave me better timing results. 124998750 is the default.

`devcfg=/sys/devices/soc0/amba/f8007000.devcfg`  
`test -d $devcfg/fclk/fclk0 || echo fclk0 > $devcfg/fclk_export`  
`echo 0 > $devcfg/fclk/fclk0/enable`  
`echo 2500000000 > $devcfg/fclk/fclk0/set_rate`  
`echo 1 > $devcfg/fclk/fclk0/enable`

In [5]:
freq = 2500000 #FPGA Clock Frequency Hz

axi_array_contents.gpio1_data = 0x02 #clear timer
axi_array_contents.gpio1_data = 0x01 #start timer
time.sleep(3) # Count to the maximim LED (8 MSB value)
axi_array_contents.gpio1_data = 0x00 #stop timer
print("Clock count: ", axi_array_contents.gpio2_data, " calculated time: ", axi_array_contents.gpio2_data/freq, " Seconds")

Clock count:  375496591  calculated time:  150.1986364  Seconds


### Test the original Shared Memory Functions
The cell below loads the default fpga bitstream and should flash the LEDs 10 times, and display the contents of the shared memory.  
For more details about the Red Pitaya Memory Map see: https://redpitaya.readthedocs.io/en/latest/developerGuide/software/build/fpga/regset_common.html

In [6]:
housekeeping_regset = np.dtype([
    ('id'          , 'uint32'),
    ('dna_lo'      , 'uint32'),
    ('dna_hi'      , 'uint32'),
    ('digital_loop', 'uint32'),
    ('ex_cd_p'     , 'uint32'),
    ('ex_cd_n'     , 'uint32'),
    ('ex_co_p'     , 'uint32'),
    ('ex_co_n'     , 'uint32'),
    ('ex_ci_p'     , 'uint32'),
    ('ex_ci_n'     , 'uint32'),
    ('reserved_2'  , 'uint32'),
    ('reserved_3'  , 'uint32'),
    ('led_control' , 'uint32')
])

In [7]:
#Test original
os.system('cat /opt/redpitaya/fpga/fpga_0.94.bit > /dev/xdevcfg')

fd = os.open('/dev/mem', os.O_RDWR)
m = mmap.mmap(fileno=fd, length=mmap.PAGESIZE, offset=0x40000000)
hk_array = np.recarray(1, housekeeping_regset, buf=m)
housekeeping_control = hk_array[0]

for i in range(10):
    housekeeping_control.led_control = 0xff
    time.sleep(0.2)
    housekeeping_control.led_control = 0x00
    time.sleep(0.2)
housekeeping_control

(1, 1928398932, 2377098, 0, 0, 0, 0, 0, 254, 255, 0, 0, 0)